In [256]:
import datetime
import itertools
import wbdata
import numpy as np
import pandas as pd
import joblib

policy = joblib.load("policy.pkl")
pair2id = {('C','C'):0, ('C','D'):1, ('D','C'):2, ('D','D'):3}
PAYOFFS = {
    ('C','C'):(3,3), ('C','D'):(0,5),
    ('D','C'):(5,0), ('D','D'):(1,1)
}

payoff_matrix = {
    ('C', 'C'): (3, 3),
    ('C', 'D'): (0, 5),
    ('D', 'C'): (5, 0),
    ('D', 'D'): (1, 1)
}


def tit_for_tat(history, player):
    """
    Cooperate on first move; thereafter mirror opponent's last move.
    history is list of tuples [(p1_move, p2_move), ...].
    player is 1 or 2.
    """
    if not history:
        return 'C'
    last = history[-1]
    last_move = ""
    if player == 1:
        last_move = last[1]
    elif player == 2:
        last_move = last[0]
    return "D" if last_move == "D" else "C"

# --- Always Cooperate ---
def always_cooperate(history, player):
    return 'C'

# --- Always Defect ---
def always_defect(history, player):
    return 'D'

# --- Grim Trigger ---
def grim_trigger(history, player):
    if not history:
        return 'C'
    opp_defected = any(
        (h[1]=='D' if player==1 else h[0]=='D')
        for h in history
    )
    return 'D' if opp_defected else 'C'


def tit_for_two_tats(history, player):
    if len(history) < 2:
        return 'C'
    last_two = history[-2:]
    opp_moves = [(h[1] if player==1 else h[0]) for h in last_two]
    return 'D' if all(m=='D' for m in opp_moves) else 'C'


def pavlov(history, player):
    """
    If last round got high payoff (3 or 5), repeat your last action; otherwise switch.
    """
    if not history:
        return 'C'
    last = history[-1]
    payoff_p1, payoff_p2 = payoff_matrix[(last[0], last[1])]
    payoff = payoff_p1 if player==1 else payoff_p2
    last_action = last[0] if player==1 else last[1]
    if payoff >= 3:
        return last_action
    return 'C' if last_action=='D' else 'D'



def soft_majority(history, player):
    """
    Cooperate if opponent has cooperated more often than defected so far; else defect.
    """
    if not history:
        return 'C'
    opp_moves = [h[1] if player==1 else h[0] for h in history]
    coop = opp_moves.count('C')
    defect = opp_moves.count('D')
    return 'C' if coop >= defect else 'D'

# --- 1. Alternator ---
def alternator(history, player):
    """
    Cooperate on odd‐numbered rounds, defect on even‐numbered rounds.
    """
    round_num = len(history) + 1
    return 'C' if round_num % 2 == 1 else 'D'


# --- 2. Suspicious Tit‑for‑Tat ---
def suspicious_tit_for_tat(history, player):
    """
    Defect on the first move, then play Tit‑for‑Tat.
    """
    if not history:
        return 'D'
    
    last = history[-1][1] if player == 1 else history[-1][0]
    return last


# --- 3. Tester ---
def tester(history, player):
    """
    Round1: Defect  
    Round2: Cooperate  
    Thereafter: Tit‑for‑Tat
    """
    if len(history) == 0:
        return 'D'
    if len(history) == 1:
        return 'C'
    last = history[-1][1] if player == 1 else history[-1][0]
    return last


# --- 4. Limited Retaliation ---
def limited_retaliation(history, player, m=2):
    """
    Cooperate until opponent defects; then defect for m rounds, then forgive.
    """
    if not history:
        return 'C'
    # extract opponent’s moves
    opp = [h[1] if player == 1 else h[0] for h in history]
    # find index of last defection
    defections = [i for i, move in enumerate(opp) if move == 'D']
    if not defections:
        return 'C'
    last_def = defections[-1]
    # soon after that defection, we punish for m rounds
    current = len(history)
    if current <= last_def + m:
        return 'D'
    return 'C'


# --- 5. Gradual (simplified) ---
def gradual(history, player):
    """
    For each opponent defection, punish with two defections, then two cooperations.
    This simplified version uses total # of opponent defections to enter a 4‐round cycle.
    """
    # count opponent defections
    opp = [h[1] if player == 1 else h[0] for h in history]
    d_count = opp.count('D')
    # cycle through: D, D, C, C
    phase = d_count % 6
    return 'D' if phase in (0, 1) else 'C'


def simulate_game(rounds, strat1, strat2):
    history = []
    for r in range(1, rounds+1):
        a1 = strat1(history, 1)
        a2 = strat2(history, 2)
        p1, p2 = payoff_matrix[(a1,a2)]
        history.append([r,a1, a2,p1,p2])

    total1 = sum(h[3] for h in history)
    total2 = sum(h[4] for h in history)
    
    if total1 > total2:
        winner = 1
    elif total2 > total1:
        winner = 2
    else:
        winner = 0

    return history, {'p1_total': total1, 'p2_total': total2}, winner

In [257]:
# --- 1. Fetch the raw tariff data again ---

countries = ["USA","CHN","DEU","BRA","IND", "JPN", "SWE", "FRA", "ITA"]
indicator = {"TM.TAX.MRCH.WM.AR.ZS": "tariff"}
date_range = (datetime.datetime(1800,1,1), datetime.datetime(2020,12,31))

raw = wbdata.get_dataframe(
    indicator,
    country=countries,
    date=date_range,
    parse_dates=True
).reset_index()

# ensure we have the columns we expect
print("Raw columns:", raw.columns.tolist())

Raw columns: ['country', 'date', 'tariff']


In [258]:
# --- 2. Discretize tariff → action (‘C’/’D’) ---

# 2a) drop any NaNs in tariff
raw = raw.dropna(subset=['tariff'])
# 2b) compute each country’s median tariff
medians = raw.groupby('country')['tariff'].median().to_dict()

# 2c) map to C/D
def to_action(row):
    return 'C' if row['tariff'] <= medians[row['country']] else 'D'

raw['action'] = raw.apply(to_action, axis=1)

print("After labeling, columns:", raw.columns.tolist())
print(raw[['country','date','tariff','action']].head())

After labeling, columns: ['country', 'date', 'tariff', 'action']
  country       date  tariff action
0  Brazil 2020-01-01    8.41      C
1  Brazil 2019-01-01    7.97      C
2  Brazil 2018-01-01    7.95      C
3  Brazil 2017-01-01    8.59      C
4  Brazil 2016-01-01    8.01      C


In [259]:
# --- 3. Pivot to get year×country action table ---
raw['year'] = raw['date'].dt.year

action_df = raw.pivot_table(
    index='year',
    columns='country',
    values='action',
    aggfunc='first'
)

print("action_df shape:", action_df.shape)
print(action_df.head())

action_df shape: (33, 9)
country Brazil China France Germany India Italy Japan Sweden United States
year                                                                      
1988       NaN   NaN    NaN     NaN   NaN   NaN     D      C           NaN
1989         D   NaN    NaN     NaN   NaN   NaN     D      D             D
1990         D   NaN    NaN     NaN     D   NaN     C    NaN             D
1991         D   NaN    NaN     NaN   NaN   NaN     D    NaN             D
1992         D     D    NaN     NaN     D   NaN     D    NaN             D


In [261]:
# --- 4. Load your policy and simulate ---

def build_features(history, seq_len):
    """
    history: list of (model_move, opp_move) tuples
    returns: 1×(2*seq_len) feature vector, padded with [0,0]
    """
    feats = []
    last = history[-seq_len:]
    pad = seq_len - len(last)
    
    # pad with zeros for missing rounds
    feats.extend([0,0] * pad)
    
    # then actual rounds
    for m, o in last:
        feats.append(int(m == 'D'))
        feats.append(int(o == 'D'))
        
    return np.array(feats).reshape(1, -1)

seq_len = 50

results = []
years     = sorted(action_df.index)
countries = list(action_df.columns)

for country in countries:
    history = []
    score_model = 0
    score_model1 = 0
    score_model2 = 0
    score_country = 0

    for year in years:
        opp = action_df.loc[year, country]
        if pd.isna(opp): continue

        X_feat = build_features(history, seq_len)  # shape (1, n_feats)
        pC, pD = policy.predict_proba(X_feat)[0]       # [P(C), P(D)]
        model_action = 'C' if pD > pC else 'D'
        
        tft_action = tit_for_tat(history, 1)
        alt_action = alternator(history, 1)

        pm, po = PAYOFFS[(model_action, opp)]
        pm1, po1 = PAYOFFS[(tft_action, opp)]
        pm2, po2 = PAYOFFS[(alt_action, opp)]
        
        score_model  += pm
        score_model1  += pm1
        score_model2  += pm2
        
        score_country+= po
        history.append((model_action, opp))


    results.append({
        'country': country,
        'model_score': score_model,
        'model1_score': score_model1,
        'model2_score': score_model2,
        'country_score': score_country,
        'score_diff': score_model - score_country
    })

res_df = pd.DataFrame(results).sort_values('model_score', ascending=False)
print(res_df.to_string(index=False))

      country  model_score  model1_score  model2_score  country_score  score_diff
        Japan          101            69            75             16          85
       Brazil           96            66            73             16          80
United States           95            65            70             15          80
        China           78            54            59             13          65
        India           77            53            58             12          65
       Sweden           71            50            53             11          60
       France           65            46            49             10          55
      Germany           65            46            49             10          55
        Italy           65            46            49             10          55


In [262]:
import datetime
import wbdata        # pip install wbdata
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# --- 1. Fetch raw tariff data (2000–2020) for a few countries ---
countries = ["USA","CHN","DEU","BRA","IND"]
indicator = {"TM.TAX.MRCH.WM.AR.ZS": "tariff"}

raw = wbdata.get_dataframe(
    indicator,
    country=countries,
    date=(datetime.datetime(1900,1,1), datetime.datetime(2020,12,31)),
    parse_dates=True
).reset_index()

# Drop rows with missing tariff
raw = raw.dropna(subset=['tariff'])

# --- 2. Label each row as 'C' or 'D' based on country median ---
medians = raw.groupby('country')['tariff'].median().to_dict()
raw['action'] = raw.apply(
    lambda row: 'C' if row['tariff'] <= medians[row['country']] else 'D',
    axis=1
)

# --- 3. Build next‑move forecasting dataset ---
def make_real_move_data(df, seq_len=5):
    """
    For each country, slide a window of length seq_len over its action series
    and predict the next action.
    Returns X (N, 2*seq_len) and y (N,).
    """
    X, y = [], []
    # ensure sorted by date
    df = df.sort_values(['country', 'date'])
    for country, group in df.groupby('country'):
        actions = group['action'].tolist()
        for i in range(len(actions) - seq_len):
            window = actions[i : i+seq_len]
            nxt    = actions[i+seq_len]
            feats = []
            # encode each round as two binary features: (you_defected, opp_defected)
            # here "you" == country, "opp" == same country since forecasting its own move
            for a in window:
                bit = int(a == 'D')
                feats.extend([bit, bit])
            X.append(feats)
            y.append(int(nxt == 'D'))
    return np.array(X), np.array(y)

# Choose sequence length
SEQ_LEN = 3
X_real, y_real = make_real_move_data(raw, seq_len=SEQ_LEN)

print("Dataset shapes:", X_real.shape, y_real.shape)

# --- 4. Train/test split ---
Xtr, Xte, ytr, yte = train_test_split(
    X_real, y_real, test_size=0.2, random_state=42, stratify=y_real
)

# --- 5. Train XGBoost ---
clf_real = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=1000,
    max_depth=3,
    learning_rate=0.1
)
clf_real.fit(Xtr, ytr)

# --- 6. Evaluate next‑move accuracy ---
y_pred = clf_real.predict(Xte)
acc = accuracy_score(yte, y_pred)
print(f"Next‑move forecast accuracy: {acc:.3f}")

Dataset shapes: (120, 6) (120,)
Next‑move forecast accuracy: 0.917


C:\Users\awast\miniconda3\envs\tf-gpu-2\lib\site-packages\xgboost\core.py:158: UserWarning: [16:47:09] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
